In [1]:
# output from prepare_pdbbind
failed_pdbs = {'1h6e': 'HIS residue (35) has the wrong set of atoms', '1svh': 'HIS residue (78) has the wrong set of atoms', '3bho': 'HIS residue (19) has the wrong set of atoms', '3bc5': 'HIS residue (6) has the wrong set of atoms', '3fxv': 'HIS residue (36) has the wrong set of atoms', '3s74': 'HIS residue (0) has the wrong set of atoms', '3smq': 'HIS residue (39) has the wrong set of atoms', '2y4k': 'HIS residue (50) has the wrong set of atoms', '3sm0': 'HIS residue (9) has the wrong set of atoms', '4hai': 'HIS residue (43) has the wrong set of atoms', '4ehm': 'HIS residue (1) has the wrong set of atoms', '4ayt': 'HIS residue (36) has the wrong set of atoms', '3rv9': 'HIS residue (21) has the wrong set of atoms', '3uib': 'HIS residue (50) has the wrong set of atoms', '4ayw': 'HIS residue (31) has the wrong set of atoms', '4gv8': 'HIS residue (34) has the wrong set of atoms', '4bae': 'HIS residue (34) has the wrong set of atoms', '4mb9': 'HIS residue (38) has the wrong set of atoms', '4ezj': 'HIS residue (18) has the wrong set of atoms', '4j8t': 'HIS residue (31) has the wrong set of atoms', '4aw8': 'HIS residue (40) has the wrong set of atoms', '4kv9': 'HIS residue (45) has the wrong set of atoms', '4kva': 'HIS residue (47) has the wrong set of atoms', '4q2k': 'HIS residue (8) has the wrong set of atoms', '4ryl': 'HIS residue (35) has the wrong set of atoms', '4rrr': 'HIS residue (48) has the wrong set of atoms', '4rrq': 'HIS residue (23) has the wrong set of atoms', '4yxu': 'HIS residue (1) has the wrong set of atoms', '4zyv': 'HIS residue (3) has the wrong set of atoms', '5iui': 'HIS residue (4) has the wrong set of atoms', '5izu': 'HIS residue (2) has the wrong set of atoms', '4xuh': 'HIS residue (12) has the wrong set of atoms', '4yb6': 'HIS residue (43) has the wrong set of atoms', '5hx8': 'HIS residue (6) has the wrong set of atoms', '5gic': 'HIS residue (48) has the wrong set of atoms', '5j3s': 'HIS residue (4) has the wrong set of atoms', '5mrd': 'HIS residue (5) has the wrong set of atoms', '5q0x': 'HIS residue (36) has the wrong set of atoms', '5ngf': 'HIS residue (41) has the wrong set of atoms', '5ien': 'HIS residue (63) has the wrong set of atoms', '5one': 'HIS residue (49) has the wrong set of atoms', '5if6': 'HIS residue (36) has the wrong set of atoms', '5wqd': 'HIS residue (21) has the wrong set of atoms', '6gbx': 'HIS residue (13) has the wrong set of atoms', '5wkl': 'HIS residue (45) has the wrong set of atoms', '6do4': 'HIS residue (54) has the wrong set of atoms', '5ypw': 'HIS residue (4) has the wrong set of atoms', '6ck3': 'HIS residue (6) has the wrong set of atoms', '5wgq': 'HIS residue (14) has the wrong set of atoms', '6do5': 'HIS residue (55) has the wrong set of atoms', '6hjk': 'HIS residue (25) has the wrong set of atoms', '6ck6': 'HIS residue (7) has the wrong set of atoms', '6cmr': 'HIS residue (10) has the wrong set of atoms', '6cjy': 'HIS residue (7) has the wrong set of atoms', '6c5t': 'HIS residue (11) has the wrong set of atoms', '6qyo': 'HIS residue (19) has the wrong set of atoms', '6i4x': 'HIS residue (42) has the wrong set of atoms', '6edl': 'HIS residue (3) has the wrong set of atoms', '6r0x': 'HIS residue (61) has the wrong set of atoms', '6e0r': 'HIS residue (4) has the wrong set of atoms', '6ebw': 'HIS residue (4) has the wrong set of atoms'}
failed_pdb_list = list(failed_pdbs.keys())
print(len(failed_pdbs))
failed_pdbs['5ypw']

61


'HIS residue (4) has the wrong set of atoms'

In [2]:
# 3nzc fails because plip does not find an interaction partner - this is also true in the web service!
# therefore I exclude it
failed_pdb_list.append('3nzc')

In [3]:
print(failed_pdb_list)

['1h6e', '1svh', '3bho', '3bc5', '3fxv', '3s74', '3smq', '2y4k', '3sm0', '4hai', '4ehm', '4ayt', '3rv9', '3uib', '4ayw', '4gv8', '4bae', '4mb9', '4ezj', '4j8t', '4aw8', '4kv9', '4kva', '4q2k', '4ryl', '4rrr', '4rrq', '4yxu', '4zyv', '5iui', '5izu', '4xuh', '4yb6', '5hx8', '5gic', '5j3s', '5mrd', '5q0x', '5ngf', '5ien', '5one', '5if6', '5wqd', '6gbx', '5wkl', '6do4', '5ypw', '6ck3', '5wgq', '6do5', '6hjk', '6ck6', '6cmr', '6cjy', '6c5t', '6qyo', '6i4x', '6edl', '6r0x', '6e0r', '6ebw', '3nzc']


In [4]:
import train
import torch
from torch import nn
from data import PDBBindInteractionDataset
from torch.utils.data import DataLoader
from e3nn import o3
from model import InteractionPredictor

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
lp_pdbbind = pd.read_csv('LP_PDBBind.csv', index_col=0)# TODO: Fix the 61 failed pdbs and remove this exclusion filter
train_pdbs = [ x for x in list(lp_pdbbind[(lp_pdbbind['new_split'] == 'train') & lp_pdbbind.CL1 & ~lp_pdbbind.covalent].index) if x not in failed_pdb_list ]
test_pdbs = [ x for x in list(lp_pdbbind[(lp_pdbbind['new_split'] == 'test') & lp_pdbbind.CL1 & ~lp_pdbbind.covalent].index) if x not in failed_pdb_list ]

# these are available after recalc
train_pdbs = ['4tln', '5tln', '2tpi', '4cts', '1rnt', '6cha', '4tmn', '1tmn', '1tlp']
test_pdbs = train_pdbs

['hbond', 'pistacking', 'hydrophobic', 'pication', 'saltbridges', 'halogenbond']
for interaction_type in ['hydrophobic']:
    print("@@@@ ", interaction_type)
    inter_pred = InteractionPredictor(n_pattern_layers = 3, 
                                    radius = 7.5,
                                    irreps_input = o3.Irreps("8x0e"),
                                    irreps_message = o3.Irreps("8x0e + 1x1o + 1x2e"),
                                    pattern_spherical_harmonics_l = 2,
                                    irreps_node = o3.Irreps("8x0e + 1x1o + 1x2e"),
                                    node_embedding_size = 8, 
                                    node_emb_hidden_layers = [], 
                                    node_act = torch.relu,
                                    edge_embedding_size = 4,
                                    edge_emb_hidden_layers = [],
                                    edge_act = torch.relu,
                                    msg_weights_hidden_layers = [16],
                                    msg_weights_act = torch.relu,
                                    node_update_hidden_layers = [24], 
                                    node_update_act = torch.relu,
                                    basis_density_per_A = 5, 
                                    inter_spherical_harmonics_l = 2,
                                    inter_tp_weights_hidden_layers = [24], 
                                    inter_tp_weights_act = torch.relu,
                                    irreps_out = o3.Irreps("1x0e"))
    print("Model weights:", sum(p.numel() for p in inter_pred.parameters() if p.requires_grad))

    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(inter_pred.parameters(), lr=1e-3, amsgrad=True)

    dataset_train = PDBBindInteractionDataset("pdbbind2020/", train_pdbs, interaction_type)
    dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True, collate_fn=dataset_train.collate_fn, pin_memory=True, num_workers=1)
    dataset_test = PDBBindInteractionDataset("pdbbind2020/", test_pdbs, interaction_type)
    dataloader_test = DataLoader(dataset_test, batch_size=4, shuffle=True, collate_fn=dataset_test.collate_fn, pin_memory=True, num_workers=1)

    print("Training size:", len(dataset_train), "Test size:", len(dataset_test))

    train.train(1000, 100, dataloader_train, dataloader_test, inter_pred, loss_fn, optimizer, save_weights=False)
    print("\n-------------------------------------------------------------\n")

@@@@  hydrophobic
Model weights: 47872
Training size: 9 Test size: 9
Using cuda device.
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    1/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    2/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    3/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    4/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    5/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    6/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    7/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    8/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [    9/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [   10/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [   11/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [   12/ 1000]
loss: 0.693147  recall:   0.00%  precision:   0.00%  [   13/ 1000]
loss: 0.693147  recall:   0.00%  precisio

In [5]:
irreps_node_in = o3.Irreps( "8x0e")
irreps_message = o3.Irreps("8x0e + 1x1o + 1x2e")
irreps_node_out = o3.Irreps("8x0e + 1x1o + 1x2e")
irreps_out = o3.Irreps("1x0e")
irreps_sh = o3.Irreps.spherical_harmonics(lmax=2)

e3tp_message_initial = o3.FullyConnectedTensorProduct(irreps_node_in,
                                              irreps_sh,
                                              irreps_message, 
                                              shared_weights=False)

e3tp_message = o3.FullyConnectedTensorProduct(irreps_node_out,
                                              irreps_sh,
                                              irreps_message, 
                                              shared_weights=False)

e3tp_node_update_initial = o3.FullyConnectedTensorProduct(irreps_node_in,
                                                  irreps_message, 
                                                  irreps_node_out,
                                                  shared_weights=False)

e3tp_node_update = o3.FullyConnectedTensorProduct(irreps_node_out,
                                                  irreps_message, 
                                                  irreps_node_out,
                                                  shared_weights=False)

e3tp_interaction = o3.FullyConnectedTensorProduct(irreps_node_out + irreps_sh, irreps_node_out, irreps_out, shared_weights=False)

Data(x=[3, 10], test=[10])